# 開始安裝

In [ ]:
#20250828執行過
%pip install ipython-sql pandas sqlalchemy

In [ ]:
%load_ext sql

In [ ]:
import sqlite3
import pandas as pd

# 資料庫路徑（根據實際位置調整）
db_path = "../SQL/mysql/database/hi.db"

# 連接到 SQLite 資料庫
conn = sqlite3.connect(db_path)


# 練習區

## import內容和DB路經

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

## 查詢

In [ ]:
# 執行查詢並用 Pandas 顯示結果  
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

query = "SELECT * FROM Daily_report WHERE tag like '%sql%';"  
df = pd.read_sql(query, conn)  
df # 顯示查詢結果

,id,content,reference,tag,created_at
0,5,新增db的tag欄位，並更新欄位資訊,SQL/mysql/database/hi.db,sql/db/schema,2025-08-28 21:50:15
1,6,時間戳變更,將時間戳更新為當地時間與當下更新的時間,sql/time,2025-08-28 21:50:15
2,7,使用jupyter寫sqlite語法,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-28 14:23:39
3,8,失敗=>使用jupyter寫sqlite语法，改用python連接db並...,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-28 14:28:42
4,9,學習如何使用 SQLite 與 Python 進行資料庫操作,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-28 14:46:34
5,13,測試文字寬度設定-已刪除id 10-12,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-29 00:19:12.031858


## 插入單筆新資料

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

# 建立 DataFrame
df = pd.DataFrame({
    'content': ['無法使用docling與markitdown'],
    'reference': ['docling已刪除'],
    'tag': ['marker/docling'],
    'created_at': [pd.Timestamp.now()]
})

# # 插入資料
# df.to_sql('Daily_report', conn, if_exists='append', index=False)

# # 全域設定（會影響整個 kernel）  
# pd.set_option('display.max_columns', None) 
# pd.set_option('display.width', 300)
# pd.set_option('display.max_colwidth', 40)   
# pd.set_option('display.expand_frame_repr', True)  
#pd.set_option('display.colheader_justify', 'left')
# 讀取並顯示最新插入的資料  
result = ("SELECT * FROM Daily_report ORDER BY id DESC LIMIT 1")  # 讀取最新一筆資料
df = pd.read_sql_query(result, conn)
df


,id,content,reference,tag,created_at
0,14,無法使用docling與markitdown,docling已刪除,marker/docling,2025-08-29 13:42:52.255496


## 插入多筆資料

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

# 建立包含多筆資料的 DataFrame
df = pd.DataFrame({
    'content': [
        '使用jupyter寫sqlite语法', 
        '學習pandas插入sqlite', 
        'SQLite資料庫操作'
    ],
    'reference': [
        'SQL/sqlite-python.ipynb', 
        'pandas/sqlite-insert.ipynb', 
        'database/sqlite-tutorial.md'
    ],
    'tag': [
        'ipython-sql/sql/', 
        'pandas/database', 
        'tutorial/sqlite'
    ],
    'created_at': [pd.Timestamp.now()]*3
})

# 插入多筆資料
df.to_sql('Daily_report', conn, if_exists='append', index=False) # append表示新增資料，不覆蓋原有資料。index=False表示不插入DataFrame的索引欄位。

# 讀取並顯示最新插入的資料
result = ("SELECT * FROM Daily_report ORDER BY id DESC LIMIT 1")  # 讀取最新一筆資料
df = pd.read_sql_query(result, conn)
df


,id,content,reference,tag,created_at
0,14,無法使用docling與markitdown,docling已刪除,marker/docling,2025-08-29 13:42:52.255496


## 刪除單筆及多筆資料

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor() # 建立游標

# 單筆刪除  
cur.execute("DELETE FROM Daily_report WHERE id = ?", (12,))  
conn.commit()

# 多筆刪除
# cur.execute("DELETE FROM Daily_report WHERE id IN (?, ?)", (4, 5))
# conn.commit()

result = pd.read_sql_query("SELECT * FROM Daily_report", conn)  # 讀取所有資料
result

,id,content,reference,tag,created_at
0,1,查詢pip中需要更新版本的套件,z-others/pip-upgrade-step.ipynb,pip/upgrade,2025-08-28 21:50:15
1,2,使用SIDERAI進行PDF解讀並轉檔py與json含RCMT-Tain...,z-my-note\hitachi-dcs\DDC-functionbl...,pdf/py/json,2025-08-28 21:50:15
2,3,學習看P&ID圖,RCMT-Tainan尚未建學習檔,P&ID/DCS,2025-08-28 21:50:15
3,4,新增MLC1的SIC圖,loop.md/尚未完成,DCS/MLC,2025-08-28 21:50:15
4,5,新增db的tag欄位，並更新欄位資訊,SQL/mysql/database/hi.db,sql/db/schema,2025-08-28 21:50:15
...,...,...,...,...,...
6,7,使用jupyter寫sqlite語法,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-28 14:23:39
7,8,失敗=>使用jupyter寫sqlite语法，改用python連接db並...,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-28 14:28:42
8,9,學習如何使用 SQLite 與 Python 進行資料庫操作,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-28 14:46:34
9,13,測試文字寬度設定-已刪除id 10-12,SQL/sqlite-python.ipynb,ipython-sql/sql/,2025-08-29 00:19:12.031858
